Main Objectives:

The primary objective of this project is to utilize time series modeling techniques to identify the top 5 zip codes for investment opportunities for a fictional real estate investment firm. The focus will predominantly be on maximizing profit margins.

Problem Statement:

The real estate investment firm has tasked us, as consultants, with identifying the top 5 zip codes for investment opportunities. In this context, "best" is defined primarily in terms of profit margins. The recommendation should prioritize zip codes with the highest potential for profitability, based on historical real estate price data.


Key Questions to Address:

1.How can profit margins be maximized through the selection of zip codes for investment?
2.Which time series modeling techniques are most suitable for predicting real estate prices and identifying profitable investment opportunities?
3.How will the recommendation incorporate considerations of risk and uncertainty associated with real estate investments?
4.What strategies can be employed to communicate the recommendation effectively to stakeholders?

Key Deliverables:

-Identification of the top 5 zip codes with the highest potential for profit margins.
-Time series models forecasting real estate prices for the selected zip codes.
-Analysis of profit potential, considering factors such as historical price trends, volatility, and investment horizon.
-Evaluation of the robustness of the recommendation and sensitivity to changes in assumptions.
-Presentation of findings through clear visualizations and a compelling narrative to facilitate decision-making by stakeholders.

In [ ]:
pip install --upgrade pip


In [ ]:
pip install pmdarima


In [ ]:
pip install --upgrade phoenix


# Import necessary libraries

In [ ]:

# Import necessary libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Ensure plots are displayed inline
%matplotlib inline

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

# Suppress specific warning
from statsmodels.tools.sm_exceptions import ConvergenceWarning
warnings.simplefilter('ignore', ConvergenceWarning)

# Set seaborn style
sns.set()

# Importing required libraries for time series analysis
from sklearn.metrics import mean_squared_error
import statsmodels.api as sm
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import pmdarima as pm
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error as calc_MSE
from math import sqrt
from matplotlib.pylab import rcParams


# Data Loading

In [ ]:
# displaying all columns
pd.set_option("display.max_columns",None)
# load the data from the csv file
df = pd.read_csv('zillow_data.csv')
# display the first 5 rows
df.head()

# Data Cleaning

In [ ]:
# creating a column of %ROI
df["%ROI"] = ((df["2018-04"] / df["2012-01"]) ** (1 / (2018-2012)) - 1) * 100


In [ ]:
# creating a column of actual ROI
df['ROI price'] = df["2018-04"] - df["2012-01"]-1

In [ ]:
# Check the columns in the dataset
df.columns


In [ ]:
# Check the shape of the dataset
df.shape

In [ ]:
# Check information about the data
df.info()

In [ ]:
# Descriptive statistics of the data
df.describe()

In [ ]:
# Convert wide format data to long format
long_data = pd.melt(df, 
                    id_vars=['RegionID', 'RegionName', 'SizeRank', 'City', 'State', 'Metro', 'CountyName', '%ROI', 'ROI price'], 
                    var_name='Date')

# Rename the RegionName column to Zipcode
long_data = long_data.rename(columns={'RegionName': 'Zipcode', 'value': 'Price'})

# Convert Zipcode to categorical data type
long_data['Zipcode'] = long_data['Zipcode'].astype('str')

# Convert Date to datetime format
long_data['Date'] = pd.to_datetime(long_data['Date'], format='%Y-%m')

long_data.head()


In [ ]:
# check columns of long data
long_data.shape

In [ ]:
# Descriptive statistics of the data
long_data.describe()

In [ ]:
# Descriptive statistics of the data
long_data.describe()

In [ ]:
# Checking for duplicates within the dataset
print(f'The number of duplicates within the dataset is: {long_data.duplicated().sum()}')


In [ ]:
# Checking for missing values
missing_percentage = long_data.isna().sum() / len(long_data) * 100
print(f'Percentage of missing values in each column:\n{missing_percentage}')


In [ ]:
# Fill missing values in the 'Metro' column with 'missing'
long_data['Metro'] = long_data['Metro'].fillna('missing')

# Checking for missing values after filling 'Metro' column
missing_percentage = long_data.isna().sum() / len(long_data) * 100
print(f'Percentage of missing values in each column after filling the "Metro" column:\n{missing_percentage}')


In [ ]:
# Statistical description of numerical variables
num_description = long_data.describe()
print("Statistical summary of numerical variables:\n", num_description)


In [ ]:
# Statistical description of categorical variables
cat_description = long_data.describe(include=['object'])
print("Statistical summary of categorical variables:\n", cat_description)


# EDA

In [ ]:
# Plotting the most popular states in the dataset
plt.figure(figsize=(12, 6))
long_data['State'].value_counts()[:10].sort_values().plot(kind="barh", color='lightgreen')
plt.xlabel("Count", fontsize=14)
plt.ylabel("US States", fontsize=14)
plt.title("Top 10 Most Popular States", fontsize=16, fontweight='bold')
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()


Carlifonia is the most popular state.

In [ ]:
# Plotting the most popular counties in the dataset
plt.figure(figsize=(12, 6))
long_data['CountyName'].value_counts()[:10].sort_values().plot(kind="barh", color='skyblue')
plt.xlabel("Count", fontsize=14)
plt.ylabel("County Name", fontsize=14)
plt.title("Top 10 Most Popular Counties", fontsize=16, fontweight='bold')
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.gca().invert_yaxis()  
plt.grid(axis='x', linestyle='--', alpha=0.7)
plt.show()


L.A. is the most popular county

In [ ]:
# Plotting the most popular cities in the dataset
plt.figure(figsize=(12, 6))
long_data['City'].value_counts()[:10].sort_values().plot(kind="barh", color='salmon')
plt.xlabel("Count", fontsize=14)
plt.ylabel("US Cities", fontsize=14)
plt.title("Top 10 Most Popular Cities", fontsize=16, fontweight='bold')
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.gca().invert_yaxis()  
plt.grid(axis='x', linestyle='--', alpha=0.7)
plt.show()


New York is the most popular city

In [ ]:
# Grouping data by mean %ROI and selecting top 30 zipcodes
grouped_roi = long_data.groupby('Zipcode')
roi_mean = grouped_roi['%ROI'].mean()
roi_mean_df = roi_mean.reset_index(name='% ROI')
roi_mean_df = roi_mean_df.sort_values(by='% ROI', ascending=False)
top_30_zipcodes_roi = roi_mean_df.head(10)


In [ ]:
# Plotting the %ROI by zipcode
plt.figure(figsize=(15, 8))
plt.bar(top_30_zipcodes_roi['Zipcode'], top_30_zipcodes_roi['% ROI'], color='skyblue')
plt.xlabel('Zipcode', fontsize=15)
plt.ylabel('% ROI', fontsize=15)
plt.title('% Return On Investment by Zipcode', fontsize=20)
plt.xticks(rotation=90)
plt.show()


According to the chart, it appears that zipcode 85035 stands out as the most lucrative area, boasting a substantial 22.8% return on investment (ROI) between 2012 and 2018.

In [ ]:
# Creating a data series to check the prices of houses over time
time_series = long_data.copy()
time_series.set_index("Date", inplace=True)
time_series = time_series["Price"]

# Plotting mean house price
plt.figure(figsize=(15, 10))
time_series.resample("A").mean().plot(color='skyblue')
plt.ylabel("Mean Price", fontsize=15)
plt.title("Mean House Price from 1996 to 2018-USA", fontsize=20)
plt.show()


This indicates an upward trend in housing prices from 1996 to 2008, followed by a sharp decline during the housing market crash. Subsequently, prices stabilized around 2012, after which they began to rise again until 2018.

# Data Preprocessing

When working with time series models, it's typically assumed that the data is stationary, meaning that its mean, variance, and autocorrelation remain constant across different time periods.

Having a stationary time series simplifies the model development process and ensures efficiency. Before proceeding with modeling, the data undergoes checks for stationarity through methods like the Dickey Fuller test and examining rolling means.

If the data is found to be non-stationary, differencing is applied to transform it into a stationary form.

In [ ]:


# Check for non-numeric values in the 'Price' column
non_numeric_values = long_data[~long_data['Price'].apply(lambda x: str(x).replace('.', '').isdigit())]

# Handle non-numeric values (replace with NaN in this example)
long_data['Price'] = pd.to_numeric(long_data['Price'], errors='coerce')

# Drop rows with NaN values in the 'Price' column
long_data.dropna(subset=['Price'], inplace=True)

# Filter data for the top 5 zipcodes based on %ROI
zipcode = long_data.sort_values('%ROI', ascending=False)['Zipcode'].unique()[:5]
top_5 = long_data[long_data['Zipcode'].isin(zipcode)]

# Group data by date and zipcode, and calculate the mean price for each group starting from 2012
grouped_5 = top_5.groupby(['Date', 'Zipcode'])['Price'].mean().reset_index()
final_df = grouped_5[grouped_5['Date'] >= "2005-01-01"]

# Check the columns present in final_df
print(final_df.columns)

# Drop unnecessary columns if they exist
columns_to_drop = ['RegionID', 'SizeRank', '%ROI', 'ROI price']
final_df.drop(columns_to_drop, axis=1, errors='ignore', inplace=True)

# Set Date as index
final_df.set_index('Date', inplace=True)


# Visualizing Home Prices:

In [ ]:
# Plotting home prices by zipcodes
colors = ['green', 'orange', 'blue', 'red', 'purple']  
line_styles = ['-', '--', '-.', ':', '-']  

for i in range(5):
    final_df[final_df['Zipcode'] == zipcode[i]].plot(y='Price', 
                                                      label=f"Zone: {zipcode[i]}", 
                                                      color=colors[i],
                                                      linestyle=line_styles[i],
                                                      figsize=(15, 8))
plt.title("Trend of Home Prices by Zone", fontsize=20)  
plt.xlabel('Year', fontsize=15)  
plt.ylabel('Price', fontsize=15) 
plt.legend()
plt.grid(True, linestyle='--', alpha=0.5)  
plt.show()


# Calculating and Visualizing Monthly Returns:

In [ ]:
# Creating a column called "ret" representing monthly returns on investment
for i in range(5):
    final_df[f'ret_{i}'] = final_df.groupby('Zipcode')['Price'].pct_change()

# Plot the monthly returns of each zipcode
for i in range(5):
    final_df.plot(y=f'ret_{i}', figsize=(15, 10), label=f"Zipcode: {zipcode[i]}")
plt.title(f'Monthly Returns per Zipcode', fontsize=20)
plt.xlabel('Date', fontsize=15)
plt.ylabel('Returns (%)', fontsize=15)
plt.legend(loc='best')
plt.show()


# Checking for Stationarity with Rolling Mean and Standard Deviation: 

In [ ]:
# Plotting rolling mean and standard deviation for each zipcode
for i in range(5):
    rollingmean = final_df[f'ret_{i}'].rolling(window=12, center=False).mean()
    rollingstd = final_df[f'ret_{i}'].rolling(window=12, center=False).std()
    fig = plt.figure(figsize=(15, 8))
    original = plt.plot(final_df[f'ret_{i}'], 
                        color=colors[i],
                        linestyle=line_styles[i],
                        label="Original")
    mean = plt.plot(rollingmean, 
                    color='black',  
                    linestyle='--',  
                    label="Rolling Mean")
    std = plt.plot(rollingstd, 
                   color='grey',  
                   linestyle='-.',  
                   label="Rolling Std")
    plt.legend(loc="best")
    plt.title(f'Rolling Mean & Std Deviation for Zone: {zipcode[i]}', fontsize=20)  #
    plt.grid(True, linestyle='--', alpha=0.5)  
    plt.show()


The above graphs reveal that certain states show non-stationarity. However, to confirm this, a Dickey Fuller test is conducted.

In [ ]:
# Create time series data for each zipcode
ts_data = {}
for zc in zipcode:
    ts_data[zc] = final_df[final_df['Zipcode'] == zc]['Price'].diff().dropna()


for zc, data in ts_data.items():
    # Perform Dickey-Fuller test
    results = adfuller(data)
    print(f'ADFuller test p-value for zipcode: {zc}')
    print('p-value:', results[1])
    
    if results[1] > 0.05:
        print('Fail to reject the null hypothesis. Data is not stationary.\n')
        # Perform differencing
        diff_data = data.diff().dropna()
        # Apply Dickey-Fuller test again
        results_diff = adfuller(diff_data)
        print(f'ADFuller test p-value after differencing for zipcode: {zc}')
        print('p-value:', results_diff[1])
        if results_diff[1] > 0.05:
            print('Fail to reject the null hypothesis even after differencing. Data might need further processing.\n')
        else:
            print('Reject the null hypothesis after differencing. Data is now stationary.\n')
    else:
        print('Reject the null hypothesis. Data is stationary.\n')


# Model Perfomance

In [ ]:
# Creating individual time series for each zip code
ts_85035 = final_df[final_df['Zipcode'] == zipcode[0]]['Price'].diff().dropna()
ts_85008 = final_df[final_df['Zipcode'] == zipcode[1]]['Price'].diff().dropna()
ts_94590 = final_df[final_df['Zipcode'] == zipcode[2]]['Price'].diff().dropna()
ts_94601 = final_df[final_df['Zipcode'] == zipcode[3]]['Price'].diff().dropna()
ts_94804 = final_df[final_df['Zipcode'] == zipcode[4]]['Price'].diff().dropna()


In [ ]:
# Define a function that plots ACF and PACF plots
def acf_pacf_plot(data, alags=40, plags=40):
    """
    Plot ACF and PACF plots for the given data.

    Parameters:
    - data: Time series data for which ACF and PACF plots are to be plotted.
    - alags: Number of lags to consider for ACF plot.
    - plags: Number of lags to consider for PACF plot.
    """
    # Create figure
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 8))
    
    # Make ACF plot
    plot_acf(data, lags=alags, zero=False, ax=ax1)
    ax1.set_title('Autocorrelation Function (ACF)')
    
    # Make PACF plot
    plot_pacf(data, lags=plags, ax=ax2)
    ax2.set_title('Partial Autocorrelation Function (PACF)')
    
    plt.tight_layout()  # Adjust layout to prevent overlap
    plt.show()


In [ ]:
# Plotting ACF and PACF for the time series data related to zipcode 85035
acf_pacf_plot(ts_85035)


In [ ]:
# Using auto arima to find the best p,d,q for the model related to zipcode 85035
model = pm.auto_arima(ts_85035, trace=True, error_action='ignore', suppress_warnings=True, stepwise=True)

# Printing the summary of the best ARIMA model
print("Summary of the best ARIMA model:")
print(model.summary())


In [ ]:
# Splitting the data into train and test sets
train_85035 = ts_85035[:'2015-01']
test_85035 = ts_85035['2015-02':]

# Fitting an ARIMA Model on the training series using parameters from the AUTO ARIMA model
# Initializing ARIMA model with the previously determined order
ARIMAmodel = ARIMA(train_85035, order=(2, 0, 1))

# Fitting the ARIMA model to the training data
ARIMAmodel = ARIMAmodel.fit()

# Printing the summary of the ARIMA model
print("Summary of the ARIMA model:")
print(ARIMAmodel.summary())

# Plotting diagnostics of the ARIMA model
ARIMAmodel.plot_diagnostics(figsize=(18, 18))
plt.show()


The residuals must exhibit no correlation and adhere to a normal distribution to meet the normality assumptions.

The QQ-plot displayed in the lower-left quadrant indicates that the residuals conform to a linear trend line, suggesting a normal distribution.

The correlogram plot in the lower-left quadrant reveals minimal correlations with lagged versions of the residuals, indicating the absence of apparent seasonality in our dataset.

The histogram portrays a bell curve, signifying that the residuals follow a normal distribution, which is favorable.

In [ ]:
# Creating a table of the upper and lower limits
pred = ARIMAmodel.get_prediction(start=pd.to_datetime('2015-02'), end=pd.to_datetime('2018-04'), dynamic=False)
pred_conf = pred.conf_int()
pred_conf.head()

In [ ]:

# Plotting the training data against the test data
plt.figure(figsize=(18, 8))

# Plot observed values
plt.plot(ts_85035.index, ts_85035, label='Observed', color="cornflowerblue")

# Plot predicted values
plt.plot(pred.predicted_mean.index, pred.predicted_mean, label='Prediction Series', alpha=0.9, color="salmon")

# Plot the range for confidence intervals
plt.fill_between(pred_conf.index, pred_conf.iloc[:, 0], pred_conf.iloc[:, 1], color='lightgray', alpha=0.5, label='Confidence Interval')

# Set axes labels and title
plt.xlabel('Date', fontsize=15)
plt.ylabel('Returns', fontsize=15)
plt.title('Testing Forecasting Model Performance', fontsize=20)
plt.legend()

plt.show()


### Model Evaluation

In [ ]:
# Calculate the RMSE for the model
rmse = mean_squared_error(test_85035, pred.predicted_mean, squared=False)
print("Root Mean Squared Error (RMSE):", rmse)

###### Forecasting for the next 5 years

In [ ]:
# Fit ARIMA model
ARIMA_MODEL = ARIMA(ts_85035, 
                    order=(2,0,1), 
                    enforce_stationarity=False, 
                    enforce_invertibility=False)

# Fit the model and print results
full_output = ARIMA_MODEL.fit()

# Print model summary
print(full_output.summary())

# Forecast for the next 5 years (60 months)
forecast = full_output.get_forecast(steps=60)

# Get the forecasted values and the confidence intervals
forecast_values = forecast.predicted_mean
confidence_intervals = forecast.conf_int()

# Print forecasted values and confidence intervals
print("Forecasted Values:")
print(forecast_values)
print("\nConfidence Intervals:")
print(confidence_intervals)


In [ ]:
# Getting a forecast for the next 60 months after the last recorded date on our dataset.
forecast = full_output.get_forecast(steps=60)
future_prediction = forecast.conf_int()
future_prediction['Price'] = forecast.predicted_mean
future_prediction.columns = ['lower','upper','prediction'] 
future_prediction.head()


In [ ]:
# Plotting our Forecast
fig, ax = plt.subplots()
ts_85035.plot(ax=ax, label='Real Values', c="blue")

future_prediction['prediction'].plot(ax=ax, label='Predicted Value', c="red")

ax.fill_between(x=future_prediction.index, y1=future_prediction['lower'], 
                y2=future_prediction['upper'], color='gray',
                label='Confidence Interval')
ax.legend() 
plt.ylabel("% Home Prices", fontsize=15)
plt.title('Average Monthly Returns - 85035 - With Forecasted Values & Confidence Intervals', fontsize=20)
plt.show()


###### A forecast for every zipcode

In [ ]:
zip_predictions = {}

# Iterate over each unique Zipcode in final_df
for zipcode in final_df['Zipcode'].unique():
    # Selecting series for the current Zipcode
    series = final_df[final_df['Zipcode'] == zipcode]['Price']
    
    # Only consider data from 2011 onwards
    recent_series = series['2011':]
    
    # Splitting the last 36 months of our series as a test dataset
    train_series = recent_series[:'2016-04']
    test_series = recent_series['2016-05':]
    
    # Auto ARIMA model
    auto_model = pm.auto_arima(train_series, 
                     trace=True,
                     error_action='ignore',
                     suppress_warnings=True,
                     stepwise=True,
                     with_intercept=False)
   
    # Plug the optimal parameter values for our Training data into a SARIMAX model that fits our entire series
    ARIMA_MODEL = SARIMAX(recent_series, 
                          order=auto_model.order, 
                          seasonal_order=auto_model.seasonal_order, 
                          enforce_stationarity=False, 
                          enforce_invertibility=False)

    # Fit the model
    output = ARIMA_MODEL.fit()

    # Getting a forecast for the next 36 months after the last recorded date on our dataset
    forecast = output.get_forecast(36)
    prediction = forecast.conf_int()
    prediction['Price'] = forecast.predicted_mean
    prediction.columns = ['lower','upper','prediction'] 
    
    # Adding the Zipcode's ROI to the zip_predictions dictionary
    zip_predictions[zipcode] = ((prediction['prediction'][-1]) - (series[-1])) / (series[-1])


In [ ]:

from statsmodels.tsa.statespace.sarimax import SARIMAX

# Initialize dictionary to store forecasted values
forecast_values = {}

# Create a for loop to forecast for every zipcode
for zipcode in final_df['Zipcode'].unique():
    # Selecting the series for the current zipcode
    series = final_df[final_df['Zipcode'] == zipcode]['Price']
    
    # Splitting the data into training and testing sets
    train_series = series[:-60]  # Use all data except the last 60 months for training
    test_series = series[-60:]   # Use the last 60 months for testing
    
    # Fit SARIMAX model
    model = SARIMAX(train_series, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
    fitted_model = model.fit()
    
    # Forecast for the next 60 months
    forecast = fitted_model.forecast(steps=60)
    
    # Store forecasted values in the dictionary
    forecast_values[zipcode] = forecast

# Convert the dictionary to a DataFrame
forecast_df = pd.DataFrame(forecast_values)

# Plot the forecasted values
plt.figure(figsize=(12, 6))
for zipcode in forecast_df.columns:
    plt.plot(forecast_df.index, forecast_df[zipcode], label=f'Zipcode: {zipcode}')
plt.title('Forecasted Prices for Next 60 Months')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.show()


##### Conclusion and Recommendation 

In [ ]:
# Get the top five zipcodes with the highest ROI
top_zipcodes = sorted(zip_predictions, key=zip_predictions.get, reverse=True)[:5]

# Create a list of ROI values for the top five zipcodes
roi_values = [zip_predictions[zipcode] for zipcode in top_zipcodes]

# Create a bar graph of the top five zipcodes and their corresponding ROI values
plt.figure(figsize=(10, 6))  # Adjust figure size as needed
bars = plt.bar(top_zipcodes, roi_values, color='orange')  # Change bar color to orange

# Add data labels to the bars
for bar, roi in zip(bars, roi_values):
    plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height(), f'{roi:.2f}', 
             ha='center', va='bottom', fontsize=12)

# Set plot labels and title
plt.xlabel('Zipcode', fontsize=15)
plt.ylabel('ROI', fontsize=15)
plt.title('Top Five Zipcodes with Highest ROI Forecast', fontsize=20)

# Add grid for better visualization
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Show plot
plt.show()
plt.gcf().canvas.draw()  # Explicitly draw the figure


Based on the graph depicted above, it's evident that the forecasted ROI for the zipcode 94804 is the highest among the top five zipcodes. Therefore, it would be advantageous for the investor to consider investing in this particular zipcode.

All Zipcodes exhibit promising forecasted prices, with positive trends, except for the 85035 zipcode.
Based on the presented graph, we derive our top five recommendations along with their anticipated ROI over a three-year period.
Consequently, the investor has the option to consider investing in any of the aforementioned zipcodes, except for 85035, which lacks a positive return on investment.